In [1]:
from services.dbconnection import DbService 
dbservice = DbService()

init


### booking a new appointment ##

*** to book an appointment we need the service and preferred date. Based on the date we will retrieve the available slots 


Assumption: The bot will be serving a specific company  eg. company_id = 3

A company has many services. To retrieve the main services pass in company_id

intent: book_appointment

Bot flow : 
- User selects a service 
- User picks a preferred date (ask the user to select a date within 7 dates from now)
- Bot retrieves available slots 
- User selects a preferred slot 
- Ask for the name 
- Confirm the appointment date and time 
- save the appointment 

In [ ]:
dbservice = DbService()
company_id = 3 
dbservice.list_main_services(company_id)

Helper functions needed to display available slots given a specific date



In [ ]:
dbservice = DbService()
from datetimerange import DateTimeRange
import datetime as dt
from datetime import datetime
from datetime import time
from errno import ETIME
DURATION = 30
def getNameOfWeek(n):
    if n == 0:
        return 'Monday'
    if n == 1:
        return 'Tuesday'
    if n == 2:
        return 'Wednesday'
    if n == 3:
        return 'Thursday'
    if n == 4:
        return 'Thursday'
    if n == 5:
        return 'Saturday'
    if n == 6:
        return 'Sunday'

def get_valid_slots_by_day(start_apt_date, days_of_week):
    day_name = getNameOfWeek(start_apt_date.weekday()).lower()
    for item in days_of_week:
        if item['title'].lower() == day_name:
            final_slots = []
            for timerange in item['timings']:
                start_date_time = datetime.combine(start_apt_date.date(), time.fromisoformat(timerange['start']))
                end_date_time = datetime.combine(start_apt_date.date(), time.fromisoformat(timerange['end']))
                while start_date_time + dt.timedelta(minutes=DURATION) <= end_date_time:
                      final_slots.append(start_date_time)
                      start_date_time = start_date_time + dt.timedelta(minutes=DURATION)

            return final_slots

    return []

def get_available_slots(date_of_interest,appointments, working_days, exception_dates, duration=30):
    
    available_slots = get_valid_slots_by_day(date_of_interest,working_days['info']['days'])
    final_slots = []
    if len(available_slots) > 0:
        for slot in available_slots:
            available_slot_start_time = slot + dt.timedelta(minutes=1)
            available_slot_end_time = slot + dt.timedelta(minutes=DURATION - 1)
            time_range1 = DateTimeRange(available_slot_start_time, available_slot_end_time)
            is_available = True
            for appointment in appointments:
                    start_time = appointment['appointment_date'] 
                    end_time =   appointment['appointment_date'] + dt.timedelta(minutes=DURATION)
                    time_range2 = DateTimeRange(start_time,end_time) 
                    if time_range1.intersection(time_range2).start_datetime != None:
                       is_available = False
                       break
                    for exc_date in exception_dates:
                        start_time = exc_date['exception_from_date'] 
                        end_time =   exc_date['exception_to_date'] 
                        time_range2 = DateTimeRange(start_time,end_time) 
                        if time_range1.intersection(time_range2).start_datetime != None:
                            is_available = False
                            break
            if is_available:
                final_slots.append(slot)
                

    return final_slots

def get_available_slots_by_date(date_of_interest, service_def_id):
    appointments = dbservice.list_valid_appointments_by_date(3,date_of_interest)
    exception_dates = dbservice.list_service_working_hrs_exception_by_date(service_def_id, date_of_interest)
    working_days = dbservice.list_service_working_days(service_def_id)
    slots = get_available_slots(date_of_interest,appointments, working_days, exception_dates, duration=30)
    return slots




Display available slots given a service definition id (each service def is unique)

In [ ]:
from services.dbconnection import DbService 

date_of_interest = datetime.strptime("25-11-2022 00:00:00",'%d-%m-%Y %H:%M:%S')
service_def_id = 3   ## belongs to company_id 3
slots = get_available_slots_by_date(date_of_interest, service_def_id)
slots

#### canceling an appointment ####
Intent : cancel_appointment

The flow: 
- Bot Retrieves valid upcoming appointments given a phone number and fromdate (usually todays date)
- The user selects and confirms the appointment to cancel 
- Update the appointment status to 3 (cancelled) 

In [ ]:
## retrieve valid appointments by phone and from date 
from services.dbconnection import DbService 
dbservice = DbService()
from datetime import datetime
from_date = datetime.today()
customer_phone = "123456"
appointments = dbservice.list_customer_valid_appointments_by_phone(from_date,customer_phone)
appointment_id = 1
status = 3 ## cancelled
dbservice.update_appointment_status(appointment_id, status)

### View upcoming appointments ### 

intent : view_appointment

- Bot Retrieves valid upcoming appointments given a phone number and fromdate (usually todays date)


In [5]:
### View upcoming appointments ### 
from services.dbconnection import DbService 
dbservice = DbService()
from datetime import datetime
from_date = datetime.today()
customer_phone = "123456"

appointments = dbservice.list_customer_valid_appointments_by_phone(from_date,customer_phone)


### View promotions ###
Bot flow
 
intent : view_promotion

display promotion_text

In [7]:
company_id = 3
dbservice.list_valid_promotions(company_id)

[{'id': 1,
  'company_id': 3,
  'title': 'تخفيضات العيد',
  'promotion_text': 'تخفيضات تصل الى 40% على جميع الخدمات لغاية نهاية الشهر',
  'active': 1}]

Insert Suggestion: 

Intent: suggestion

- save the suggestion to db 

In [ ]:
dbservice = DbService()

dbservice.insert_suggestion(3,"nadir","123456","the waiting room is very crowdedS")